# jsats3D Project Notebook
jsats3d was developed by Kleinschmidt to assist researchers with cleaning and managing small scale acoustic telemetry studies (<1000 tagged individuals, <20 receivers)  The resulting dataset represents a standardized, repeatable and defensible product.  This notebook carries out every task one may have when analyzing acoustic telemetry data, including: creating a standardized technology agnostic project database, importing raw data directly from receiver downloads, identifying and removing multipath, synchronizing clocks, positioning fish in 3D space, and visualizing resultant point clouds with kernel density functions.  In future iterations of the software, we will replace the functional sample scripts with a Jupyter Notebook to guide users through training, classification, and data preparation.

Acoustic telemetry projects create vast quantities of data.  To handle the amount of data generated by these projects, jsats3d relies on a SQLite project database.  SQLite is an in-process library that implements a self-contained, server-less, zero configuration, transactional SQL database engine (SQLite, 2017).  More importantly, SQLite can handle simultaneous reads, so it is well-suited for write-once, read-many largescale data analysis projects that employ parallel processing across multiple cores.  To view the SQLite project database download either: [sqlite browser](http://sqlitebrowser.org/) or [sqlite studio](https://sqlitestudio.pl/index.rvt) .  Both SQLite viewers can execute SQL statements, meaning you can create a query and export to csv file from within the database viewer.  They are not that different from Microsoft Access, users familiar with databases will have an easy transition into SQLite.

The software is written in Python 3.7.x and uses dependencies outside of the standard packages.  Please make sure you have the following modules installed and updated when running telemetry analysis: Numpy, Pandas, Networkx, Matplotlib, Sqlite3, Statsmodels, and SciKit Learn.  The software also uses a number of standard packages including: Time, Math, Os, Datetime, Operator, and Collections.  

The project notebook will guide the end user through a coplete acoustic telemetry project.  However, you could import jsats3d into your own proprietary scripts and data management routines.  The provided scripts and notebooks are examples only and represent current best practices 

---

# Project Set Up

## Supporting Python Modules
jsats3d can't do it alone, we need other libraries to assist with data I/O, analysis, and visualization.

In [13]:
import os
import sys
import pandas as pd
import numpy as np

up to you, but you can repress warnings by running the following cell

In [14]:
import warnings
warnings.filterwarnings('ignore')

---

## Connecting to and using jsats3d
The author of this notebook recommends users clone jsats3d into a local directory of their choosing.  

In [15]:
jsats_dir = r"C:\Users\knebiolo\OneDrive - Kleinschmidt Associates, Inc\Software\jsats3d"
sys.path.append(jsats_dir)
import jsats3d

---

## Identify Project Parameters and Create a Project Workspace
Important project parameters include input workspaces for raw data and final data products and software output.  Parameters also include a database name, conversion from UTC to the local timezone, a benchmark elevation in meters, a master receiver used for time keeping, the coordinate system surveyed points were collected in, a project syncronization time, and a project end time.  

**The project syncronization time is a critical parameter.  It is the moment the master receiver clock was syncronized to UTC - aka GPS time.**

In [16]:
utc_conv = -7                           # hours from UTC to local
bm_elev = 861.5/3.28084                 # benchmark elevation
bm_elev_units = 'meters'                # elevation units
masterReceiver = 'R05'                  # identify the master receiver -aka the study metronome
EPSG = 26910                            # coordinate system 
output_units = 'meters'                 # output units
synch_start_time = "2018-06-07 14:09"   # study synchronization time
synch_time_end = "2018-08-09 10:30"     # study end time


Before proceeding to the next step, the end user of this notebook must create the following folder structure:

* Project_Name
    * Data *for storing raw data and project set up files*
	* Output *for storing figures, modeling files, and scratch data*
	    * Scratch *holds intermediate files that are managed by the software* 
		* Figures *holds figures exported by the software*

**Do not alter the structure of the directory.** The notebook expects the project directory is structured exactly as depicted.


Now we need to connect the project workspace to this notebook.

In [17]:
inputWS = r"C:\Users\knebiolo\Desktop\jsats_Notebook_Test\Data"
outputWS = r"C:\Users\knebiolo\Desktop\jsats_Notebook_Test\Output"

---

## Create a Project Database
the following commands will create a manufacturer agnostic project database

In [18]:
dbName = 'jsats_notebook_test.db'            # give your project a name
dbDir = os.path.join(outputWS,dbName)
jsats3d.create_project_db(outputWS, dbName) # create project database
print ('project database created')

project database created


---

## Master Receiver Table
The receiver file must contain the following fields and should be saved as a comma delimited text file in the project Data folder.  Please see the schema below.  Please save the master receiver file to the ‘Data’ folder.    A good name for this file is “tblReceiver.csv”.  Note, the Z coordinate is optional because it may be impractical to accurately measure receivers at depth.

| Field      | Data Type |                                      Comment                             |
|------------|-----------|--------------------------------------------------------------------------|
|Rec_ID      |String     |(required) - unique alphanumeric receiver ID                              |
|Type        |String     |(required) - receiver make and model                                      |
|Tag_ID      |String     |(required) - alphanumeric ID for receiver's beacon tag                    |
|Ref_Elev    |String     |(required) - receivers either relative to 'WSEL' or 'BM'                  |
|X           |Float      |(required) - X surveyed coordinate of receiver in study EPSG              |
|Y           |Float      |(required) - Y surveyed coordinate of reciever in study EPSG              |
|Z           |Float      |(optional) - Z surveyed coordinate of receiver in study Vertical Datum    |
|X_t         |Float      |(required) - transformed X coordinate into local datum                    |
|Y_t         |Float      |(required) - transformed Y coordinate into local datum                    |
|Z_t         |Float      |(optional) - transformed Z coordinate into local datum                    |

Import the receiver table and write it to the project database

In [19]:
jsats3d.study_data_import(pd.read_csv(os.path.join(inputWS,'tblReceiver.csv')),dbDir,'tblReceiver')  

---

## Master Tag Table
The master tag file contains a list of all active acoustic tags in the project and the table must contain the following fields and should be saved as a comma delimited text file in the project Data folder. 

Please name the file  “tblTag.csv” and save in the project's Data folder.  

Please see the schema below. 

| Field      | Data Type |                                      Comment                             |
|------------|-----------|--------------------------------------------------------------------------|
|Tag_ID      |String     |(required) - unique alphanumeric tag ID                                   |
|pulseRate   |Float      |(required) - tag burst rate in decimal seconds                            |
|TagType     |String     |(required) - a tag can either be 'study' or 'beacon'                      |


In [20]:
jsats3d.study_data_import(pd.read_csv(os.path.join(inputWS,'tblTag.csv')),dbDir,'tblTag')

---
## Temperature Table
In fresh water, sound travels faster than 1 km per second and is affected by temperature.  The warmer the temperature, the faster the speed of sound.  Unfortunately, an acoustic pulse from a tag may travel through a stratified water column, changing velocity as it approaches the receiver.  A variable speed of sound means positional accuracy will be reduced.  It is important to have continuous temperature readings of the entire water column at different locations within the forebay.  The final temperature at time is the median of all temperature readings.  The more stratification, the more positional error.

Please name the temperature file 'tblTemp.csv' and place in the project's Data folder.

The temperature file for use in jsats3d must contain the following fields:

| Field      | Data Type |                                      Comment                             |
|------------|-----------|--------------------------------------------------------------------------|
|location    |String     |(required) - location wihtin study area                                   |
|depth_ft    |Float      |(required) - depth of temperature sensor                                  |
|meas_dt     |DateTime   |(required) - a date and time of temperature reading                       |
|temp_f      |Float      |(required) - a temperature reading in Farhenheight                        |

In [21]:
jsats3d.study_data_import(pd.read_csv(os.path.join(inputWS,'tblTemp.csv'), parse_dates = True),dbDir,'tblTemp')

---
## Import Water Surface Elevation

Three of the receivers in the original source study tracked with water surface elevation or WSEL.  Therefore, it was necessary to obtain operations data that tracked WSEL in time.  

The WSEL file for use in jsats3d must contain the following fields:

| Field      | Data Type |                                      Comment                             |
|------------|-----------|--------------------------------------------------------------------------|
|timeStamp   |String     |(required) - date time precise to the decimal second                      |
|WSEL        |Float      |(required) - water surface elevation in project's vertical datum          |

In [22]:
jsats3d.study_data_import(pd.read_csv(os.path.join(inputWS,'tblWSEL.csv'), parse_dates = True),dbDir,'tblWSEL')

Write Study Parameters to Database

In [23]:
jsats3d.set_study_parameters(utc_conv,
                             bm_elev,
                             bm_elev_units,
                             output_units,
                             masterReceiver,
                             synch_start_time,
                             synch_time_end,dbDir)

---
## Import Raw Data
The data generated by the source project was managed with separate directories for each receiver.  jsats3d imports data for a single receiver at a time.  

The function acoustic_data_import requires the end user to declare the Receiver ID, Receiver Type, data directory, and database directory

In [24]:
jsats3d.acoustic_data_import('R01','Teknologic',os.path.join(inputWS, "Aggregated","R01"),dbDir)  #R01
jsats3d.acoustic_data_import('R02','Teknologic',os.path.join(inputWS, "Aggregated","R02"),dbDir)  #R02
jsats3d.acoustic_data_import('R03','Teknologic',os.path.join(inputWS, "Aggregated","R03"),dbDir)  #R03
jsats3d.acoustic_data_import('R04','Teknologic',os.path.join(inputWS, "Aggregated","R04"),dbDir)  #R04
jsats3d.acoustic_data_import('R05','Teknologic',os.path.join(inputWS, "Aggregated","R05"),dbDir)  #R05
jsats3d.acoustic_data_import('R06','Teknologic',os.path.join(inputWS, "Aggregated","R06"),dbDir)  #R06
jsats3d.acoustic_data_import('R07','Teknologic',os.path.join(inputWS, "Aggregated","R07"),dbDir)  #R07
jsats3d.acoustic_data_import('R08','Teknologic',os.path.join(inputWS, "Aggregated","R08"),dbDir)  #R08
jsats3d.acoustic_data_import('R09','Teknologic',os.path.join(inputWS, "Aggregated","R09"),dbDir)  #R09

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\knebiolo\\Desktop\\jsats_Notebook_Test\\Data\\Aggregated\\R01'